# PoP NBr12-14

In [ ]:
from WS_Mdl import utils as Utl
from WS_Mdl import geo
# import os
# import pandas as pd
# import numpy as np
# from tqdm import tqdm
# from pathlib import Path

In [12]:
import importlib
importlib.reload(geo)
importlib.reload(Utl)

<module 'WS_Mdl.utils' from 'c:\\users\\karam014\\onedrive - universiteit utrecht\\ws_mdl\\code\\WS_Mdl\\utils.py'>

# Options

In [ ]:
l_MdlN = [f'NBr{i}' for i in range(12, 14+1)]

# Calculate Mo AVG HDs maps for each L of a Sim

In [3]:
for MdlN in l_MdlN:
    geo.HD_IDF_Mo_Avg_to_MBTIF(MdlN)

*** NBr12 *** - 2010-01 ✔  - Elapsed time (from start): 0:00:36.307957
*** NBr12 *** - 2010-02 ✔  - Elapsed time (from start): 0:00:36.307957
*** NBr12 *** - 2010-03 ✔  - Elapsed time (from start): 0:00:38.173563
*** NBr12 *** - 2010-04 ✔  - Elapsed time (from start): 0:00:38.173563
*** NBr12 *** - 2010-05 ✔  - Elapsed time (from start): 0:00:38.173563
*** NBr12 *** - 2010-06 ✔  - Elapsed time (from start): 0:00:39.583908
*** NBr12 *** - 2010-07 ✔  - Elapsed time (from start): 0:00:39.583908
*** NBr12 *** - 2010-08 ✔  - Elapsed time (from start): 0:00:39.583908
*** NBr12 *** - 2010-09 ✔  - Elapsed time (from start): 0:00:39.583908
*** NBr12 *** - 2010-10 ✔  - Elapsed time (from start): 0:00:39.583908
*** NBr12 *** - 2010-11 ✔  - Elapsed time (from start): 0:01:12.155375
*** NBr12 *** - 2010-12 ✔  - Elapsed time (from start): 0:01:16.377405
*** NBr12 *** - 2011-01 ✔  - Elapsed time (from start): 0:01:16.490125
*** NBr12 *** - 2011-02 ✔  - Elapsed time (from start): 0:01:16.490125
*** NB

# Calculate differences

# GXG

In [13]:
for MdlN in l_MdlN:
    geo.HD_IDF_GXG_to_MBTIF(MdlN)

	 GXG_L1 ✔ - Elapsed time (from start): 0:02:51.963043
	 GXG_L2 ✔ - Elapsed time (from start): 0:02:51.998189
	 GXG_L3 ✔ - Elapsed time (from start): 0:02:52.830019
	 GXG_L4 ✔ - Elapsed time (from start): 0:02:54.011512
	 GXG_L5 ✔ - Elapsed time (from start): 0:02:54.042823
	 GXG_L6 ✔ - Elapsed time (from start): 0:02:54.044842
	 GXG_L7 ✔ - Elapsed time (from start): 0:02:54.044842
	 GXG_L8 ✔ - Elapsed time (from start): 0:02:54.044842
	 GXG_L9 ✔ - Elapsed time (from start): 0:02:54.044842
	 GXG_L10 ✔ - Elapsed time (from start): 0:02:54.044842
	 GXG_L11 ✔ - Elapsed time (from start): 0:05:23.029238
	 GXG_L12 ✔ - Elapsed time (from start): 0:05:23.030246
	 GXG_L13 ✔ - Elapsed time (from start): 0:05:31.562264
	 GXG_L14 ✔ - Elapsed time (from start): 0:05:35.752696
	 GXG_L15 ✔ - Elapsed time (from start): 0:05:35.762265
	 GXG_L16 ✔ - Elapsed time (from start): 0:05:35.762265
	 GXG_L17 ✔ - Elapsed time (from start): 0:05:38.266863
	 GXG_L18 ✔ - Elapsed time (from start): 0:05:40.219459
	